In [87]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from perlin_noise import PerlinNoise
import math
from PIL import Image

In [88]:
index = 0
map_no = 2
xpix, ypix = 256, 256

def generate(xpix,ypix,seed,octaves,multipliers,power=1):
    assert(len(octaves) > 0 and len(multipliers) >= len(octaves))
    m = np.zeros((xpix,ypix,1))
    noises = [PerlinNoise(octaves=octave,seed=seed) for octave in octaves]
    for x in range(xpix):
        for y in range(ypix):
            noise_val = 0
            for noise,multiplier in zip(noises,multipliers):
                noise_val += multiplier * noise([x/xpix,y/ypix])
            m[x,y] = pow(noise_val / (sum(multipliers)),power)
    return m

def normalize(xpix,ypix,m):
    min_val, max_val = np.min(m), np.max(m)

    for x in range(xpix):
        for y in range(ypix):
            normalized_val = (m[x,y] - min_val) / (max_val - min_val)
            m[x,y] = normalized_val
    return m

def islandify(xpix,ypix,m,coeff=0.5):
    assert(coeff >= 0 and coeff <= 1)

    for x in range(xpix):
        for y in range(ypix):
            nx = 2 * x / xpix - 1
            ny = 2 * y / ypix - 1
            
            # d = pow((abs(nx) + abs(ny)),1)
            d = math.sqrt((nx) ** 2 + (ny) ** 2)

            falloff = max(0,1 - d * coeff)
            m[x,y] = m[x,y] * pow(falloff,2)

            # d = (1-pow(nx,2)) * (1-pow(ny,2))
            # m[x,y] = (1-coeff) * m[x,y] + coeff * d
    return m

def islandify_constrain(xpix,ypix,m,coeff=0.5,constrain=0.9):
    assert(coeff >= 0 and coeff <= 1)
    for x in range(xpix):
        for y in range(ypix):
            nx = 2 * x / xpix - 1
            ny = 2 * y / ypix - 1
            xp = (constrain-pow(nx,2))
            yp = (constrain-pow(ny,2))

            if (xp < 0): xp = 0
            if (yp < 0): yp = 0

            d = xp * yp
            m[x,y] = (1-coeff) * m[x,y] + coeff * d
            
    return m

def crop(m,w,h,margin_y,margin_x):
    min_val = np.amin(m)
    nm = np.array(m)
    for i in range(w):
        for j in range(h):
            if not (i > margin_x and i < w - margin_x):
                nm[i,j] = [min_val]
                continue
            if not (j > margin_y and j < h - margin_x):
                nm[i,j] = [min_val]
                continue
    return nm

In [93]:
for i in range(15):
    m = generate(xpix,ypix,i+3,[8,16],[1,0.125],power=1)
    m = normalize(xpix,ypix,m)
    m2 = np.array(m)
    m2 = islandify(xpix,ypix,m2,1)
    # m2 = crop(m2,xpix,ypix,10,10)
    m2 = (255*(m2 - np.min(m2))/np.ptp(m2)).astype(np.uint8)
    np_img = np.squeeze(m2, axis=2)
    im = Image.fromarray(np_img,mode="L")
    im.save("data/perlin/perlin%d.png" % i)
    # fig, (ax1, ax2) = plt.subplots(1,2)
    # ax1.imshow(m,cmap="gray")
    # ax2.imshow(m2,cmap="gray")
    # plt.show()